In [38]:
from pathlib import Path
import pandas as pd
import numpy as np

path = "./Measurement Data"

# Project

## 1. Merge every trip dataset into a global one

In [39]:
files = Path(path).rglob("*.csv")

In [40]:
def rename_columns(df):

    # New column names
    new_columns = {
        'Time [s]': 'time_s',
        'Velocity [km/h]': 'velocity_km_h',
        'Velocity [km/h]]]': 'velocity_km_h',
        'Elevation [m]': 'elevation_m',
        'Throttle [%]': 'throttle_percent',
        'Motor Torque [Nm]': 'motor_torque_nm',
        'Longitudinal Acceleration [m/s^2]': 'longitudinal_acceleration_m_s2',
        'Regenerative Braking Signal ': 'regenerative_braking_signal',
        'Battery Voltage [V]': 'battery_voltage_v',
        'Battery Current [A]': 'battery_current_a',
        'Battery Temperature [°C]': 'battery_temperature_c',
        'max. Battery Temperature [°C]': 'max_battery_temperature_c',
        'SoC [%]': 'soc_percent',
        'displayed SoC [%]': 'displayed_soc_percent',
        'min. SoC [%]': 'min_soc_percent',
        'max. SoC [%)': 'max_soc_percent',
        'Heating Power CAN [kW]': 'heating_power_can_kw',
        'Heating Power LIN [W]': 'heating_power_lin_w',
        'Requested Heating Power [W]': 'requested_heating_power_w',
        'AirCon Power [kW]': 'aircon_power_kw',
        'Heater Signal': 'heater_signal',
        'Heater Voltage [V]': 'heater_voltage_v',
        'Heater Current [A]': 'heater_current_a',
        'Ambient Temperature [°C]': 'ambient_temperature_c',
        'Coolant Temperature Heatercore [°C]': 'coolant_temperature_heatercore_c',
        'Requested Coolant Temperature [°C]': 'requested_coolant_temperature_c',
        'Coolant Temperature Inlet [°C]': 'coolant_temperature_inlet_c',
        'Heat Exchanger Temperature [°C]': 'heat_exchanger_temperature_c',
        'Cabin Temperature Sensor [°C]': 'cabin_temperature_sensor_c',
        'Ambient Temperature Sensor [°C]': 'ambient_temperature_sensor_c',
        'Coolant Volume Flow +500 [l/h]': 'coolant_volume_flow_plus_500_l_h',
        'Temperature Coolant Heater Inlet [°C]': 'temperature_coolant_heater_inlet_c',
        'Temperature Coolant Heater Outlet [°C]': 'temperature_coolant_heater_outlet_c',
        'Temperature Heat Exchanger Outlet [°C]': 'temperature_heat_exchanger_outlet_c',
        'Temperature Defrost lateral left [°C]': 'temperature_defrost_lateral_left_c',
        'Temperature Defrost lateral right [°C]': 'temperature_defrost_lateral_right_c',
        'Temperature Defrost central [°C]': 'temperature_defrost_central_c',
        'Temperature Defrost central left [°C]': 'temperature_defrost_central_left_c',
        'Temperature Defrost central right [°C]': 'temperature_defrost_central_right_c',
        'Temperature Footweel Driver [°C]': 'temperature_footweel_driver_c',
        'Temperature Footweel Co-Driver [°C]': 'temperature_footweel_co_driver_c',
        'Temperature Feetvent Co-Driver [°C]': 'temperature_feetvent_co_driver_c',
        'Temperature Feetvent Driver [°C]': 'temperature_feetvent_driver_c',
        'Temperature Head Co-Driver [°C]': 'temperature_head_co_driver_c',
        'Temperature Head Driver [°C]': 'temperature_head_driver_c',
        'Temperature Vent right [°C] ': 'temperature_vent_right_c_1',
        'Temperature Vent right [°C]': 'temperature_vent_right_c_2',
        'Temperature Vent central right [°C]': 'temperature_vent_central_right_c',
        'Temperature Vent central left [°C]': 'temperature_vent_central_left_c',
    }

    # Rename columns
    df.rename(columns=new_columns, inplace=True)

    # Drop useless columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    return df

In [41]:
dfs = list()

# Process each file
for file in files:
    print(file, " Importing...")
    data = pd.read_csv(file, encoding='ISO-8859-13', delimiter=';')
    data['trip_file_name'] = file.stem
    data = rename_columns(data)
    dfs.append(data)

# Concatenate all dataframes
df = pd.concat(dfs, ignore_index=True)

Measurement Data/TripA32.csv  Importing...
Measurement Data/TripB02.csv  Importing...
Measurement Data/TripA10.csv  Importing...
Measurement Data/TripA23.csv  Importing...
Measurement Data/TripB24.csv  Importing...
Measurement Data/TripB38.csv  Importing...
Measurement Data/TripB12.csv  Importing...


Measurement Data/TripB21.csv  Importing...
Measurement Data/TripB19.csv  Importing...
Measurement Data/TripA12.csv  Importing...
Measurement Data/TripB20.csv  Importing...
Measurement Data/TripA17.csv  Importing...
Measurement Data/TripA13.csv  Importing...
Measurement Data/TripA27.csv  Importing...
Measurement Data/TripA29.csv  Importing...
Measurement Data/TripB32.csv  Importing...
Measurement Data/TripB13.csv  Importing...
Measurement Data/TripB09.csv  Importing...
Measurement Data/TripB10.csv  Importing...
Measurement Data/TripB26.csv  Importing...
Measurement Data/TripB11.csv  Importing...
Measurement Data/TripB17.csv  Importing...
Measurement Data/TripA31.csv  Importing...
Measurement Data/TripB35.csv  Importing...
Measurement Data/TripA20.csv  Importing...
Measurement Data/TripB36.csv  Importing...
Measurement Data/TripB28.csv  Importing...
Measurement Data/TripB30.csv  Importing...
Measurement Data/TripA04.csv  Importing...
Measurement Data/TripB15.csv  Importing...
Measurement

In [42]:
# Enregistrer le fichier
df.to_csv('01_Mergerd.csv', index=False)

## 2. Extracting data from the travel_name column
The travel_name column contains informations about the trip. We will extract the following information:
  - The trip season (A: Summer, B: Winter)
  - The trip identification number

In [43]:
df['trip_season'] = np.where(df['trip_file_name'].str.contains('A'), 'summer', 'winter')

# print(df['trip_season'].value_counts())

df['trip_id'] = df['trip_file_name'].str.removeprefix('Trip')

# print(df['trip_id'].value_counts())

## 3. Handling missing data
> The measurement data are divided into two categories. Category A was recorded in summer and does not contain all measured data due to trouble with the measurement system.  Category B was recorded in winter and is consistent. 

In [44]:
print(df.isnull().sum().sort_values()[df.isnull().sum() > 0])

requested_coolant_temperature_c         19829
soc_percent                             30793
min_soc_percent                         30793
displayed_soc_percent                   30794
max_soc_percent                         30794
coolant_temperature_inlet_c            310587
heater_current_a                       310587
heater_voltage_v                       310587
heating_power_lin_w                    310587
coolant_temperature_heatercore_c       310587
temperature_vent_right_c_2             472340
temperature_vent_central_left_c        472340
temperature_vent_central_right_c       472340
temperature_vent_right_c_1             472340
temperature_head_driver_c              472340
temperature_head_co_driver_c           472340
temperature_feetvent_driver_c          472340
temperature_feetvent_co_driver_c       472340
temperature_footweel_co_driver_c       472340
temperature_defrost_central_right_c    472340
temperature_defrost_central_left_c     472340
temperature_defrost_central_c     

We can choose to:
  - Drop columns with missing data
  - Drop rows with missing data
  - Fill missing data with statistical methods (mean, median, mode, etc.)

Drop rows is not a good idea because we will lose a lot of data (all the summer data).

The missing data for the "summer" trips are mostly precisions about temperatures for the cabin. We can fill them with the global values or forget them and focus on the global cabin data.

---
### State of Charge (SoC) missing data
**For the folowing columns :**
- soc_percent
- min_soc_percent
- displayed_soc_percent
- max_soc_percent

The missing value occurs only for the first and last occurence of each trip (start and end of the trip). We can fill them with the next (for the first occurence) or previous (for the last occurence) value because we can assume that the SoC (State of Charge) doesn't change during the first and last 0.1s of the trip.

In [45]:
# For each trip, fill missing values (for theses columns) with the previous (if exists) or next (if exists) value.

df['soc_percent'] = df.groupby('trip_id')['soc_percent'].ffill().bfill()
df['min_soc_percent'] = df.groupby('trip_id')['min_soc_percent'].ffill().bfill()
df['displayed_soc_percent'] = df.groupby('trip_id')['displayed_soc_percent'].ffill().bfill()
df['max_soc_percent'] = df.groupby('trip_id')['max_soc_percent'].ffill().bfill()

print(df.isnull().sum().sort_values()[df.isnull().sum() > 0])

requested_coolant_temperature_c         19829
coolant_temperature_inlet_c            310587
heater_current_a                       310587
heater_voltage_v                       310587
heating_power_lin_w                    310587
coolant_temperature_heatercore_c       310587
temperature_vent_right_c_2             472340
temperature_vent_central_left_c        472340
temperature_vent_central_right_c       472340
temperature_vent_right_c_1             472340
temperature_head_driver_c              472340
temperature_head_co_driver_c           472340
temperature_feetvent_driver_c          472340
temperature_feetvent_co_driver_c       472340
temperature_footweel_co_driver_c       472340
temperature_defrost_central_right_c    472340
temperature_defrost_central_left_c     472340
temperature_defrost_central_c          472340
temperature_defrost_lateral_right_c    472340
temperature_defrost_lateral_left_c     472340
temperature_heat_exchanger_outlet_c    472340
temperature_coolant_heater_outlet_

----
### Cabin temperature missing data
For the cabin temperatures, we choose to drop the columns because, all these precisions are not relevant for the analysis of the trips. We will focus on the global cabin temperature.

In [46]:
columns_to_drop = [
    'temperature_vent_right_c_1',
    'temperature_vent_right_c_2',
    'temperature_vent_central_left_c',
    'temperature_vent_central_right_c',
    'temperature_head_driver_c',
    'temperature_head_co_driver_c',
    'temperature_feetvent_driver_c',
    'temperature_feetvent_co_driver_c',
    'temperature_footweel_driver_c',
    'temperature_footweel_co_driver_c',
    'temperature_defrost_central_right_c',
    'temperature_defrost_central_left_c',
    'temperature_defrost_central_c',
    'temperature_defrost_lateral_right_c',
    'temperature_defrost_lateral_left_c'
]

df.drop(columns=columns_to_drop, inplace=True)

print(df.isnull().sum().sort_values()[df.isnull().sum() > 0])

requested_coolant_temperature_c         19829
heater_voltage_v                       310587
heater_current_a                       310587
coolant_temperature_heatercore_c       310587
coolant_temperature_inlet_c            310587
heating_power_lin_w                    310587
ambient_temperature_sensor_c           472340
temperature_coolant_heater_inlet_c     472340
temperature_coolant_heater_outlet_c    472340
temperature_heat_exchanger_outlet_c    472340
coolant_volume_flow_plus_500_l_h       496034
dtype: int64


----
### Heating/A.C. missing data
For the heater data (current, voltage), we choose to use the mean value for the voltage, and calculate the current with the formula: P = U * I, where P is the power ("heating_power_can_kw"), U the voltage and I the current. We can calculate the current with the formula: I = P / U.

In [52]:
# fill missing values with the mean of the column, for the same season
df['heater_voltage_v'] = df['heater_voltage_v'].fillna(df.groupby('trip_season')['heater_voltage_v'].transform('mean'))

# use I=P/U to calculate the missing values of the heater_current_a
df['heater_current_a'] = df['heater_current_a'].fillna((df['heating_power_can_kw'] * 1000) / df['heater_voltage_v'])

print(df.isnull().sum().sort_values()[df.isnull().sum() > 0])

coolant_temperature_heatercore_c       310587
coolant_temperature_inlet_c            310587
temperature_coolant_heater_inlet_c     472340
temperature_coolant_heater_outlet_c    472340
temperature_heat_exchanger_outlet_c    472340
coolant_volume_flow_plus_500_l_h       496034
dtype: int64


For the **heating_power_lin_w**, we already have the heating_power_can_kW that is consistent. We can drop the heating_power_lin_w column.

In [48]:
df.drop(columns=["heating_power_lin_w"], inplace=True)

For the **requested_coolant_temperature**, we can't find correlations with other columns, and don't vary during the trips (always at 0, 85 or 215). We think that this column is not relevant for the analysis of the trips. We will drop it.

In [49]:
df.drop(columns=["requested_coolant_temperature_c"], inplace=True)

For **ambiant_temperature_sensor_c**, a column named `ambient_temperature` and it's consistent. We can drop the `ambiant_temperature_sensor_c` column.

In [50]:
df.drop(columns=["ambient_temperature_sensor_c"], inplace=True)

# 4. Colums description
## General
- 'trip_file_name'
- 'trip_id'


## Environment
- 'time_s',
- 'ambient_temperature_c'
- 'trip_season'
- 'elevation_m'

## Longitudinal dynamics and drive train
- 'velocity_km_h'
- 'longitudinal_acceleration_m_s2'
- 'throttle_percent'
- 'motor_torque_nm'
- 'regenerative_braking_signal'

## Battery
- 'battery_voltage_v'
- 'battery_current_a'
- 'battery_temperature_c'
- 'max_battery_temperature_c'
- 'soc_percent'
- 'displayed_soc_percent'
- 'min_soc_percent'
- 'max_soc_percent'

## Cabin
- 'cabin_temperature_sensor_c'


## High voltage heater
- 'heater_signal'
- 'heater_voltage_v'
- 'heater_current_a'
- 'requested_heating_power_w'
- 'heating_power_can_kw'
- 'temperature_coolant_heater_inlet_c'
- 'coolant_temperature_heatercore_c'
- 'temperature_coolant_heater_outlet_c'


## Auxiliaries
### Air conditioning
- 'aircon_power_kw'

## Other
- 'coolant_temperature_inlet_c',
- 'heat_exchanger_temperature_c'
- 'temperature_heat_exchanger_outlet_c'
- 'coolant_volume_flow_plus_500_l_h'

![UML](image.png)